In [1]:
# Reload the notebook if an external file is updated
%load_ext autoreload
%autoreload 2

import sys

from pathlib import Path

path = str(Path.cwd().parent)
sys.path.append(path)

In [2]:
import numpy as np

from constant import SETTINGS
from copy import deepcopy
from datatype.dataset import Dataset
from datatype.settings import Settings
from datatype.spectrogram import Linear, Mel, Spectrogram
from io import BytesIO
from itertools import permutations
from math import sqrt
from PIL import Image, ImageDraw, ImageEnhance, ImageFilter, ImageFont, ImageOps
from wand.image import Image as ImageMagick

In [3]:
def create_grid(collection, text):
    column = 5
    padding = 10

    row = len(collection) // column

    if len(collection) % column:
        row = row + 1

    width, height = np.vectorize(lambda x: x.size)(collection)
    maximum_width = np.amax(width) + 10
    maximum_height = np.amax(height) + 10

    if maximum_width < 500:
        maximum_width = 500

    page_width = maximum_width * column + (padding * column) - padding
    page_height = maximum_height * row + (padding * row) - padding

    grid = Image.new(
        'L',
        color='white',
        size=(page_width, page_height),
    )

    x = 0
    y = 0

    for index, (image, duration) in enumerate(zip(collection, text), 0):
        x_offset = int(
            (maximum_width - image.width) / 2
        )

        y_offset = int(
            (maximum_height - image.height) / 2
        )
        
#         draw = ImageDraw.Draw(image)
#         font = ImageFont.truetype('fonts/arial.ttf', 32)
#         text_width = draw.textlength(str(duration), font=font)

#         size = (
#             (image.width - text_width) // 2,
#             y_offset
#         )

#         draw.text(
#             size,
#             str(duration),
#             (0),
#             font=font
#         )

        grid.paste(
            image,
            (x + x_offset, y + y_offset)
        )

        x = x + maximum_width + padding

        if (index + 1) % column == 0:
            y = y + maximum_height + padding
            x = 0

    grid = ImageOps.expand(
        grid,
        border=(20, 100),
        fill=(255)
    )

    return grid

In [4]:
def create_image(spectrogram):
    buffer = BytesIO()
    image = Image.fromarray(~spectrogram)
    
    return image

In [5]:
def create_spectrogram(signal, settings):
    spectrogram = Spectrogram()
    strategy = Mel(signal, settings)
    spectrogram.strategy = strategy

    return spectrogram.generate()

In [6]:
dataset = Dataset('segment')
dataframe = dataset.load()

dataframe = dataframe[~dataframe.exclude]

In [7]:
path = SETTINGS.joinpath('spectrogram.json')
settings = Settings.from_file(path)

In [12]:
dataframe['duration'] = dataframe['duration'].astype(float)
# subset = dataframe.nlargest(100, 'duration')
subset = dataframe.nsmallest(200, 'duration')

In [15]:
subset.duration

17764    0.000998
17777    0.000998
17790    0.000998
17795    0.000998
17798    0.000998
           ...   
8888     0.011973
8896     0.011973
9245     0.011973
9524     0.011973
9531     0.011973
Name: duration, Length: 200, dtype: float64

In [14]:
segment = subset.segment.to_numpy()
duration = subset.duration.to_numpy()
duration = np.around(duration, decimals=4)

spectrogram = np.frompyfunc(
    lambda x, y: create_spectrogram(x, y),
    2,
    1
)(segment, settings)

image = np.frompyfunc(
    lambda x: create_image(x),
    1,
    1
)(spectrogram)

grid = create_grid(image, duration)
grid.show()

E:\code\personal\warbler.py\venv\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=512 is too small for input signal of length=44
  return f(*args, **kwargs)
e:\code\personal\warbler.py\warbler.py\datatype\spectrogram.py:66: RuntimeWarning: invalid value encountered in divide
  data = (self.data - minimum) / (maximum - minimum)
E:\code\personal\warbler.py\venv\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=512 is too small for input signal of length=45
  return f(*args, **kwargs)
E:\code\personal\warbler.py\venv\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=512 is too small for input signal of length=88
  return f(*args, **kwargs)
E:\code\personal\warbler.py\venv\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=512 is too small for input signal of length=132
  return f(*args, **kwargs)
E:\code\personal\warbler.py\venv\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=512 is too small for i